<a href="https://colab.research.google.com/github/fenzhantw/Dacon_jobcare_recommend/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 모듈 불러오기

In [1]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 1.1 MB/s 


In [2]:
!pip install optuna

     |████████████████████████████████| 308 kB 7.5 MB/s 
     |████████████████████████████████| 209 kB 70.3 MB/s 
     |████████████████████████████████| 80 kB 11.8 MB/s 
     |████████████████████████████████| 75 kB 6.0 MB/s 
     |████████████████████████████████| 149 kB 77.3 MB/s 
     |████████████████████████████████| 49 kB 9.2 MB/s 
     |████████████████████████████████| 112 kB 85.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=f0f9fbb938716d46dba958a1da912e4284ea162ff82d14cf27b5e4ecfc63a663
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings, random
warnings.filterwarnings(action='ignore')

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from sklearn.metrics import f1_score 
from sklearn.feature_extraction import FeatureHasher

from sklearn.cluster import KMeans
from lightgbm import LGBMClassifier 
from catboost import Pool,CatBoostClassifier

##함수화 작업

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/Colab Notebooks/job_care/'


#분류 추가
def add_code(df, d_code, h_code, l_code):
    df = df.copy()   

    # D Code
    df['person_prefer_d_1_n'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_1_s'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_1_m'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_1_l'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_2_n'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_2_s'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_2_m'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_2_l'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_3_n'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_3_s'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_3_m'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_3_l'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['contents_attribute_d_n'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['contents_attribute_d_s'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['contents_attribute_d_m'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['contents_attribute_d_l'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    # H Code
    df['person_prefer_h_1_m'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['person_prefer_h_2_m'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['person_prefer_h_3_m'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['contents_attribute_h_m'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    df['person_prefer_h_1_l'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_2_l'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_3_l'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['contents_attribute_h_l'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 대분류코드'])

    # L Code
    df['contents_attribute_l_n'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 세분류코드'])
    df['contents_attribute_l_s'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 소분류코드'])
    df['contents_attribute_l_m'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 중분류코드'])
    df['contents_attribute_l_l'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 대분류코드'])
    return df
 

def drop_features(df):
  df = df.drop(['id','person_rn','contents_rn','person_prefer_f','person_prefer_g'],axis=1)
  return df

def get_date(df):
  for i in range(2):
    df[i].contents_open_dt = pd.to_datetime(df[i].contents_open_dt)

    df[i]['month'] = df[i].contents_open_dt.dt.month
    df[i]['day'] = df[i].contents_open_dt.dt.day
    df[i]['week'] = df[i].contents_open_dt.dt.isocalendar().week
    df[i]['dayofweek'] = df[i].contents_open_dt.dt.dayofweek
    df[i]['hour'] = df[i].contents_open_dt.dt.hour
  #  df[i]['minute'] = df[i].contents_open_dt.dt.minute

    df[i].drop(['id', 'contents_open_dt'], axis=1, inplace=True)

def show_hist_by_target(df,columns):
  cond_1 = (df['target'] == 1)
  cond_0 = (df['target'] == 0)

  for column in columns:
    fix,axs = plt.subplots(nrows=1,ncols=2,figsize = (12, 4),squeeze=False)
    sns.violinplot(x='target',y=column,data=df,ax=axs[0][0])
    sns.distplot(df[cond_0][column],ax=axs[0][1],label='0',color='blue')
    sns.distplot(df[cond_1][column],ax=axs[0][1],label='1',color='red')

# 10개 이상 카테고리 얻기
def get_data_up10(df):
  up10_cat = []
  columns_names=df.columns.values
  for i in columns_names:
    if df[i].value_counts().count() >=10:
      up10_cat.append(i)

  up10_cat.append('target')
  up10_cat.remove('contents_open_dt')

  df2 = df[up10_cat]
  train_target_1 = df[df['target']==1]
  train_target_0 = df[df['target']==0]
  
  return train_target_1,train_target_0,up10_cat

# 데이터 비율 차이 구하기
def get_data_dict(train_target_1,train_target_0,up10_cat):
  list_diff=[]
  for i in up10_cat[:-1]:
    serise=abs((train_target_1[i].value_counts()/train_target_1[i].shape[0])*100 - (train_target_0[i].value_counts()/train_target_0[i].shape[0])*100)
    temp_list=serise[serise>=0.5].index.tolist()
    list_diff.append(temp_list)

  dict1 = {up10_cat[i]:list_diff[i] for i in range(len(up10_cat[:-1])) if len(list_diff[i])>1 }
  return dict1

# 데이터 비율의 차이가 0.5보다 크면 1 아니면 0
def get_data_1_or_0(train,test,dict1):
  for data in [train,test]:
    for key,values in dict1.items():
      data.loc[:, f"tar_enc_{key}"] = 0
      for i in values:
        con=data[data[key]==i].index
        data.loc[con, f"tar_enc_{key}"] =1
  return train,test

# 기존의 0.5이상인 컬럼 값은 들고오고,이하인것만 0으로 된 컬럼 추가
def get_data_data_or_0(train,test,dict1):
  for data in [train,test]:
    for key,values in dict1.items():
      data.loc[:, f"tar_enc_div_{key}"] = 0
      for i in values:
        con=data[data[key]==i].index
        data.loc[con, f"tar_enc_div_{key}"] = i  
  return train,test

# 10개 이하 카테고리 얻기

def get_data_down10(df):
  col_name = list(df.columns)
  object_nunique = list(map(lambda col: df[col].nunique(), col_name))
  d = dict(zip(col_name, object_nunique))
  d = pd.DataFrame([col_name, object_nunique]).T
  onehot_list = list(d[(d[1]>2)&(d[1]<=10)][0])

  return onehot_list

# 범주 갯수가 10개 이하인 변수에 대해서 타겟값이 0.5이상이면 1 아니면 0인 변수 추가/ 범주 갯수가 10개 이하인 변수에 대해서 타겟값이 0.55이상이면 1, 0.45~0.55사이면 2, 0.45 이하이면 3인 변수 추가

def get_down10_1_or_0(train,test,onehot_list):
  for col in onehot_list:
    temp_df = []
        
    # 명목형 변수에서 각 값 별로 타겟값의 평균을 대입
    feat = train.groupby(col)["target"].agg("mean")
    feat = feat.to_dict()
    test.loc[:, f"tar_enc_{col}"] = X_test[col].map(feat)
    temp_df.append(test)

    temp_train_feat = train[col].map(feat)
    temp_test_feat = test[col].map(feat)

    train.loc[:, f"tar_enc_{col}"] = temp_train_feat
    test.loc[:, f"tar_enc_{col}"] = temp_test_feat
    train.drop('target', axis=1, inplace=True)

    return train,test,temp_df

def div_value_2(temp):
    new_value = ''

    if temp >=0.5 : new_value = 1
    else : new_value = 0
    
    return new_value


def div_value_3(temp):
    new_value = ''

    if temp >=0.55 : new_value = 1
    elif temp <=0.45 : new_value = -1
    else : new_value = 0
    
    return new_value

def get_down10_1_or_00(train,X_train,test,onehot_list):
    div_value = list(train.iloc[:,X_train.shape[1]:X_train.shape[1]+abs(X_train.shape[1] - train.shape[1])].columns)

    for i in range(len(div_value)):
      X_train.loc[:, f"tar_div2_{onehot_list[i]}"] = train.loc[:,div_value[i]].apply(lambda x : div_value_2(x))
      test.loc[:, f"tar_div2_{onehot_list[i]}"] = test.loc[:, div_value[i]].apply(lambda x : div_value_2(x))
      
    for i in range(len(div_value)):
      X_train.loc[:, f"tar_div3_{onehot_list[i]}"] = train.loc[:, div_value[i]].apply(lambda x : div_value_3(x))
      test.loc[:, f"tar_div3_{onehot_list[i]}"] = test.loc[:, div_value[i]].apply(lambda x : div_value_3(x))

    return X_train,test



Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
def drop_features(df):
  df = df.drop(['id','person_rn','contents_rn','person_prefer_f','person_prefer_g'],axis=1)
  return df

def get_date(df):
  for i in range(2):
    df[i].contents_open_dt = pd.to_datetime(df[i].contents_open_dt)

    df[i]['month'] = df[i].contents_open_dt.dt.month
    df[i]['day'] = df[i].contents_open_dt.dt.day
    df[i]['week'] = df[i].contents_open_dt.dt.isocalendar().week
    df[i]['dayofweek'] = df[i].contents_open_dt.dt.dayofweek
    df[i]['hour'] = df[i].contents_open_dt.dt.hour
  #  df[i]['minute'] = df[i].contents_open_dt.dt.minute

    df[i].drop(['id', 'contents_open_dt'], axis=1, inplace=True)

def show_hist_by_target(df,columns):
  cond_1 = (df['target'] == 1)
  cond_0 = (df['target'] == 0)

  for column in columns:
    fix,axs = plt.subplots(nrows=1,ncols=2,figsize = (12, 4),squeeze=False)
    sns.violinplot(x='target',y=column,data=df,ax=axs[0][0])
    sns.distplot(df[cond_0][column],ax=axs[0][1],label='0',color='blue')
    sns.distplot(df[cond_1][column],ax=axs[0][1],label='1',color='red')


In [ ]:
  train = pd.read_csv(path + "train.csv")
  test = pd.read_csv(path + "test.csv")
  submission = pd.read_csv(path + "sample_submission.csv")
  d_code = pd.read_csv(path + '속성_D_코드.csv', index_col=0).T.to_dict()
  h_code = pd.read_csv(path + '속성_H_코드.csv', index_col=0).T.to_dict()
  l_code = pd.read_csv(path + '속성_L_코드.csv', index_col=0).T.to_dict()

In [ ]:
# 10개 이상 카테고리 얻기
def get_data_up10(df):
  up10_cat = []
  columns_names=df.columns.values
  for i in columns_names:
    if df[i].value_counts().count() >=10:
      up10_cat.append(i)

  up10_cat.append('target')
  up10_cat.remove('contents_open_dt')

  df2 = df[up10_cat]
  train_target_1 = df[df['target']==1]
  train_target_0 = df[df['target']==0]
  
  return train_target_1,train_target_0,up10_cat

# 데이터 비율 차이 구하기
def get_data_dict(train_target_1,train_target_0,up10_cat):
  list_diff=[]
  for i in up10_cat[:-1]:
    serise=abs((train_target_1[i].value_counts()/train_target_1[i].shape[0])*100 - (train_target_0[i].value_counts()/train_target_0[i].shape[0])*100)
    temp_list=serise[serise>=0.5].index.tolist()
    list_diff.append(temp_list)

  dict1 = {up10_cat[i]:list_diff[i] for i in range(len(up10_cat[:-1])) if len(list_diff[i])>1 }
  return dict1

# 데이터 비율의 차이가 0.5보다 크면 1 아니면 0
def get_data_1_or_0(train,test,dict1):
  for data in [train,test]:
    for key,values in dict1.items():
      data.loc[:, f"tar_enc_{key}"] = 0
      for i in values:
        con=data[data[key]==i].index
        data.loc[con, f"tar_enc_{key}"] =1
  return train,test

# 기존의 0.5이상인 컬럼 값은 들고오고,이하인것만 0으로 된 컬럼 추가
def get_data_data_or_0(train,test,dict1):
  for data in [train,test]:
    for key,values in dict1.items():
      data.loc[:, f"tar_enc_div_{key}"] = 0
      for i in values:
        con=data[data[key]==i].index
        data.loc[con, f"tar_enc_div_{key}"] = i  
  return train,test


In [ ]:
# 10개 이하 카테고리 얻기

def get_data_down10(df):
  col_name = list(df.columns)
  object_nunique = list(map(lambda col: df[col].nunique(), col_name))
  d = dict(zip(col_name, object_nunique))
  d = pd.DataFrame([col_name, object_nunique]).T
  onehot_list = list(d[(d[1]>2)&(d[1]<=10)][0])

  return onehot_list

# 범주 갯수가 10개 이하인 변수에 대해서 타겟값이 0.5이상이면 1 아니면 0인 변수 추가/ 범주 갯수가 10개 이하인 변수에 대해서 타겟값이 0.55이상이면 1, 0.45~0.55사이면 2, 0.45 이하이면 3인 변수 추가

def get_down10_1_or_0(train,test,onehot_list):
  for col in onehot_list:
    temp_df = []
        
    # 명목형 변수에서 각 값 별로 타겟값의 평균을 대입
    feat = train.groupby(col)["target"].agg("mean")
    feat = feat.to_dict()
    test.loc[:, f"tar_enc_{col}"] = X_test[col].map(feat)
    temp_df.append(test)

    temp_train_feat = train[col].map(feat)
    temp_test_feat = test[col].map(feat)

    train.loc[:, f"tar_enc_{col}"] = temp_train_feat
    test.loc[:, f"tar_enc_{col}"] = temp_test_feat
    train.drop('target', axis=1, inplace=True)

    return train,test,temp_df

def div_value_2(temp):
    new_value = ''

    if temp >=0.5 : new_value = 1
    else : new_value = 0
    
    return new_value


def div_value_3(temp):
    new_value = ''

    if temp >=0.55 : new_value = 1
    elif temp <=0.45 : new_value = -1
    else : new_value = 0
    
    return new_value

def get_down10_1_or_00(train,X_train,test,onehot_list):
    div_value = list(train.iloc[:,X_train.shape[1]:X_train.shape[1]+abs(X_train.shape[1] - train.shape[1])].columns)

    for i in range(len(div_value)):
      X_train.loc[:, f"tar_div2_{onehot_list[i]}"] = train.loc[:,div_value[i]].apply(lambda x : div_value_2(x))
      test.loc[:, f"tar_div2_{onehot_list[i]}"] = test.loc[:, div_value[i]].apply(lambda x : div_value_2(x))
      
    for i in range(len(div_value)):
      X_train.loc[:, f"tar_div3_{onehot_list[i]}"] = train.loc[:, div_value[i]].apply(lambda x : div_value_3(x))
      test.loc[:, f"tar_div3_{onehot_list[i]}"] = test.loc[:, div_value[i]].apply(lambda x : div_value_3(x))

    return X_train,test


In [ ]:
train = add_code(train, d_code, h_code, l_code)
train = drop_features(train)
test = add_code(test, d_code, h_code, l_code)
test = drop_features(test)
y_train = train.loc[:,'target']
X_train = train.drop(['target'],axis=1)
train_target_1,train_target_0,up10_cat = get_data_up10(train)
dict1 = get_data_dict(train_target_1,train_target_0,up10_cat)
train,test = get_data_1_or_0(train,test,dict1)
train,test = get_data_data_or_0(train,test,dict1)


## 시작

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/Colab Notebooks/job_care/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
train = pd.read_csv(path + "train.csv")
X_test = pd.read_csv(path + "test.csv")
submission = pd.read_csv(path + "sample_submission.csv")

d_code = pd.read_csv(path + '속성_D_코드.csv', index_col=0)
h_code = pd.read_csv(path + '속성_H_코드.csv', index_col=0)
l_code = pd.read_csv(path + '속성_L_코드.csv', index_col=0)

In [ ]:
d_code = d_code.T.to_dict()
h_code = h_code.T.to_dict()
l_code = l_code.T.to_dict()

In [ ]:
def add_code(df, d_code, h_code, l_code):
    df = df.copy()   

    # D Code
    df['person_prefer_d_1_n'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_1_s'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_1_m'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_1_l'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_2_n'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_2_s'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_2_m'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_2_l'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_3_n'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_3_s'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_3_m'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_3_l'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['contents_attribute_d_n'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['contents_attribute_d_s'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['contents_attribute_d_m'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['contents_attribute_d_l'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    # H Code
    df['person_prefer_h_1_m'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['person_prefer_h_2_m'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['person_prefer_h_3_m'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['contents_attribute_h_m'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    df['person_prefer_h_1_l'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_2_l'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_3_l'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['contents_attribute_h_l'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 대분류코드'])

    # L Code
    df['contents_attribute_l_n'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 세분류코드'])
    df['contents_attribute_l_s'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 소분류코드'])
    df['contents_attribute_l_m'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 중분류코드'])
    df['contents_attribute_l_l'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 대분류코드'])
    return df

train = add_code(train, d_code, h_code, l_code)
X_test = add_code(X_test, d_code, h_code, l_code)
print("train_data.shape: ", train.shape)
print("test_data.shape: ", X_test.shape)

train_data.shape:  (501951, 63)
test_data.shape:  (46404, 62)


## 컬럼 추가

In [ ]:
data = [train, X_test]

for i in range(2):
  data[i].contents_open_dt = pd.to_datetime(data[i].contents_open_dt)

  data[i]['month'] = data[i].contents_open_dt.dt.month
  data[i]['day'] = data[i].contents_open_dt.dt.day
  data[i]['week'] = data[i].contents_open_dt.dt.isocalendar().week
  data[i]['dayofweek'] = data[i].contents_open_dt.dt.dayofweek
  data[i]['hour'] = data[i].contents_open_dt.dt.hour
#  data[i]['minute'] = data[i].contents_open_dt.dt.minute

  data[i].drop(['id', 'contents_open_dt'], axis=1, inplace=True)

In [ ]:
X_train = train.drop('target',axis=1)
y_train = train['target']

In [ ]:
X_train = X_train.drop(['person_rn', 'contents_rn', 'person_prefer_f', 'person_prefer_g'],axis=1)
X_test = X_test.drop(['person_rn', 'contents_rn', 'person_prefer_f', 'person_prefer_g'],axis=1)

In [ ]:
col_name = list(X_train.columns)

In [ ]:
object_nunique = list(map(lambda col: X_train[col].nunique(), col_name))
d = dict(zip(col_name, object_nunique))

# # Print number of unique entries by column, in ascending order
# sorted(d.items(), key=lambda x: x[1])

In [ ]:
d = pd.DataFrame([col_name, object_nunique]).T
onehot_list = list(d[(d[1]>2)&(d[1]<=10)][0])

In [ ]:
df = pd.concat([X_train, y_train], axis=1)
df_test = X_test.copy()

for col in onehot_list:
    # 명목형 변수에서 각 값 별로 타겟값의 평균을 대입
    feat = df.groupby(col)["target"].agg("mean")
    feat = feat.to_dict()

    # 명목형 변수에 매칭되는 각 평균값을 temp_train/test_feat로 생성
    temp_train_feat = df[col].map(feat)
    temp_test_feat = df_test[col].map(feat)

    #temp_train/test_feat를 tar_enc_{col} 형태로 컬럼 생성
    df.loc[:, f"tar_enc_{col}"] = temp_train_feat
    df_test.loc[:, f"tar_enc_{col}"] = temp_test_feat

# df.drop('target', axis=1, inplace=True)

In [ ]:
#분기 함수 생성
def div_value_2(temp):
    new_value = ''

    if temp >=0.5 : new_value = 1
    else : new_value = 0
    
    return new_value


def div_value_3(temp):
    new_value = ''

    if temp >=0.55 : new_value = 1
    elif temp <=0.45 : new_value = -1
    else : new_value = 0
    
    return new_value

In [ ]:
#위에서 생성된 새로운 컬럼 div_value에 저장
div_value = list(df.iloc[:,X_train.shape[1]:X_train.shape[1]+abs(X_train.shape[1] - df.shape[1])].columns)

#lambda 함수로 위에 생성한 분기값 입력(div_value_2)
for i in range(len(div_value)):
  X_train.loc[:, f"tar_div2_{onehot_list[i]}"] = df.loc[:,div_value[i]].apply(lambda x : div_value_2(x))
  X_test.loc[:, f"tar_div2_{onehot_list[i]}"] = df_test.loc[:, div_value[i]].apply(lambda x : div_value_2(x))

#lambda 함수로 위에 생성한 분기값 입력(div_value_3)
for i in range(len(div_value)):
  X_train.loc[:, f"tar_div3_{onehot_list[i]}"] = df.loc[:, div_value[i]].apply(lambda x : div_value_3(x))
  X_test.loc[:, f"tar_div3_{onehot_list[i]}"] = df_test.loc[:, div_value[i]].apply(lambda x : div_value_3(x))

In [ ]:
# 하나의 값만 가지는 컬럼 drop
drop_feature = X_train.columns[X_train.nunique() < 2].tolist()

X_train.drop(drop_feature,axis=1, inplace=True)
X_test.drop(drop_feature,axis=1, inplace=True)

In [ ]:
# 10개 이상 카테고리 출력
up10_cat = list(d[d[1]>10][0])
up10_cat

['person_prefer_d_1',
 'person_prefer_d_2',
 'person_prefer_d_3',
 'person_prefer_e',
 'person_prefer_h_1',
 'person_prefer_h_2',
 'person_prefer_h_3',
 'contents_attribute_l',
 'contents_attribute_d',
 'contents_attribute_e',
 'contents_attribute_h',
 'person_prefer_d_1_n',
 'person_prefer_d_1_s',
 'person_prefer_d_1_m',
 'person_prefer_d_1_l',
 'person_prefer_d_2_n',
 'person_prefer_d_2_s',
 'person_prefer_d_2_m',
 'person_prefer_d_2_l',
 'person_prefer_d_3_n',
 'person_prefer_d_3_s',
 'person_prefer_d_3_m',
 'person_prefer_d_3_l',
 'contents_attribute_d_n',
 'contents_attribute_d_s',
 'contents_attribute_d_m',
 'contents_attribute_d_l',
 'person_prefer_h_1_m',
 'person_prefer_h_2_m',
 'person_prefer_h_3_m',
 'contents_attribute_h_m',
 'person_prefer_h_1_l',
 'person_prefer_h_2_l',
 'person_prefer_h_3_l',
 'contents_attribute_h_l',
 'contents_attribute_l_n',
 'contents_attribute_l_s',
 'contents_attribute_l_m',
 'contents_attribute_l_l',
 'month',
 'day',
 'week',
 'hour']

In [ ]:
if onehot_list in  up10_cat:
  print(True)
else: print(False)

False


In [ ]:
train_up = df[up10_cat]
test_up  = X_test[up10_cat]

train_up['target'] = y_train

In [ ]:
train_1 = train_up[train_up['target']==1]
train_0 = train_up[train_up['target']==0]

In [ ]:
list_diff=[]
for i in up10_cat[:-1]:
  serise=abs((train_1[i].value_counts()/train_1[i].shape[0])*100 - (train_0[i].value_counts()/train_0[i].shape[0])*100)
  temp_list=serise[serise>=0.5].index.tolist()
  list_diff.append(temp_list)

# 0.5차이 나는 값이 1개 이상인 카테고리 딕셔너리로 담기 / 0.5 차이가 나는 변수가 하나인 컬럼은 dict에서 제외
dict1 = {up10_cat[i]:list_diff[i] for i in range(len(up10_cat[:-1])) if len(list_diff[i])>1 }

In [ ]:
# 컬럼 추가
for data in [train_up,test_up]:
  for key,values in dict1.items():
    data.loc[:, f"tar_enc_1_10_div_{key}"] = 0
    for i in values:
      con=data[data[key]==i].index
      data.loc[con, f"tar_enc_1_10_div_{key}"] = 1

In [ ]:
#기존의 0.5이상인 컬럼 값은 들고오고, 이하인것만 0으로 된 컬럼 추가
for data in [train_up,test_up]:
  for key,values in dict1.items():
    data.loc[:, f"tar_enc_2_10_div_{key}"] = 0
    for i in values:
      con=data[data[key]==i].index
      data.loc[con, f"tar_enc_2_10_div_{key}"] = i

In [ ]:
X_train = pd.concat([X_train, train_up.iloc[:,44:]], axis=1)
X_test = pd.concat([X_test, test_up.iloc[:,43:]], axis=1)

##비지도 학습으로 그룹 변수 추가

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42).fit(X_train)
X_train['cluster_5'] = kmeans.predict(X_train)
X_test['cluster_5'] = kmeans.predict(X_test)

In [ ]:
kmeans = KMeans(n_clusters=10, random_state=42).fit(X_train)
X_train['cluster_10'] = kmeans.predict(X_train)
X_test['cluster_10'] = kmeans.predict(X_test)

In [ ]:
kmeans = KMeans(n_clusters=20, random_state=42).fit(X_train)
X_train['cluster_20'] = kmeans.predict(X_train)
X_test['cluster_20'] = kmeans.predict(X_test)

In [ ]:
kmeans = KMeans(n_clusters=30, random_state=42).fit(X_train)
X_train['cluster_30'] = kmeans.predict(X_train)
X_test['cluster_30'] = kmeans.predict(X_test)

In [ ]:
kmeans = KMeans(n_clusters=40, random_state=42).fit(X_train)
X_train['cluster_40'] = kmeans.predict(X_train)
X_test['cluster_40'] = kmeans.predict(X_test)

In [ ]:
kmeans = KMeans(n_clusters=50, random_state=42).fit(X_train)
X_train['cluster_50'] = kmeans.predict(X_train)
X_test['cluster_50'] = kmeans.predict(X_test)

## 최종 데이터셋 출력

In [ ]:
X_train.to_csv(path + "X_train_last.csv", index=False)
X_test.to_csv(path + "X_test_last.csv", index=False)

In [31]:
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/Colab Notebooks/job_care/'

X_train = pd.read_csv(path + "X_train_last.csv")
X_test = pd.read_csv(path + "X_test_last.csv")
train = pd.read_csv(path + "train.csv")
y_train = train["target"]
submission = pd.read_csv(path +"sample_submission.csv")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Origin Model(catboost)

- OOF Ensemble



In [ ]:
n_est = 3000
seed = 42
n_fold = 10
n_class = 1

X = X_train.copy()
y = y_train

In [ ]:
#k 개의  모델을 다 사용하되 앙상블 처럼 각 모델에서 주는 output 들의 평균값을 사용하여 최종 pred 산출(OOF Ensemble)

skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = 0
cat_cols = X_train.columns[X_train.nunique() >= 2].tolist()
for fold in range(n_fold):
  print(f'\n----------------- Fold {fold} -----------------\n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

  model_cat = CatBoostClassifier(eval_metric="F1", task_type='GPU')
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
  
  cat_pred[valid_idx] = pd.DataFrame(model_cat.predict_proba(X_valid)[:, 1], index=X_valid.index)
  cat_pred_test = cat_pred_test + (model_cat.predict_proba(X_test)[:, 1] / n_fold)
  pred = np.where(cat_pred >= 0.4 , 1, 0)
  pred_test = np.where(cat_pred_test >= 0.4 , 1, 0)

  print(f'CV F1 score: {f1_score(y_valid, pred[valid_idx]):.6f}')
    
print(f'\tF1 score: {f1_score(y, pred):.6f}')

In [ ]:
submission['target'] = pred_test
submission

,id,target
0,0,0
1,1,0
2,2,1
3,3,0
4,4,1
...,...,...
46399,46399,1
46400,46400,1
46401,46401,1
46402,46402,1


In [ ]:
submission.to_csv(path + "prediction_0123_2.csv", index=False)

# Model 2(catboost)

- 하이퍼 파라미터 적용, k =10




In [ ]:
import optuna

In [ ]:
def objective(trial, data=X, target=y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 16),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.005, 0.02, 0.05, 0.08, 0.1]),
        'n_estimators': trial.suggest_int('n_estimators', 2000, 8000),
        'max_bin': trial.suggest_int('max_bin', 200, 400),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 300),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0001, 1.0, log = True),
        'subsample': trial.suggest_float('subsample', 0.1, 0.8),
        'random_seed': 42,
        'task_type': 'GPU',
        'loss_function': 'Logloss',
        'eval_metric': 'F1',
        'bootstrap_type': 'Poisson'
    }
    
    model = CatBoostClassifier(**params)  
    model.fit(X_train, y_train, eval_set = [(X_val,y_val)], early_stopping_rounds = 222, verbose = False)
    cat_pred = model.predict_proba(X_val)[:,1]
    y_pred = np.where(cat_pred >= 0.4 , 1, 0)
    F1_score = f1_score(y_val, y_pred)

    return F1_score

In [ ]:
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best value:', study.best_value)

Number of finished trials: 30

Best trial: {'max_depth': 10, 'learning_rate': 0.02, 'n_estimators': 2099, 'max_bin': 231, 'min_data_in_leaf': 61, 'l2_leaf_reg': 0.21933100943955486, 'subsample': 0.6525918984479465}

Best value: 0.6913040754315071

In [ ]:
seed = 42
n_fold = 7
n_class = 1

X = X_train.copy()
y = y_train

In [ ]:
#k 개의  모델을 다 사용하되 앙상블 처럼 각 모델에서 주는 output 들의 평균값을 사용하여 최종 pred 산출(OOF Ensemble)

skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = 0
cat_cols = X_train.columns[X_train.nunique() >= 2].tolist()
for fold in range(n_fold):
  print(f'\n----------------- Fold {fold} -----------------\n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

  model_cat = CatBoostClassifier(max_depth=10,learning_rate=0.02,n_estimators=2099,max_bin=231,min_data_in_leaf=61,l2_leaf_reg=0.2193,subsample=0.6525,eval_metric="F1", task_type='GPU',bootstrap_type='Poisson')
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
  
  cat_pred[valid_idx] = pd.DataFrame(model_cat.predict_proba(X_valid)[:, 1], index=X_valid.index)
  cat_pred_test = cat_pred_test + (model_cat.predict_proba(X_test)[:, 1] / n_fold)
  pred = np.where(cat_pred >= 0.4 , 1, 0)
  pred_test = np.where(cat_pred_test >= 0.4 , 1, 0)

  print(f'CV F1 score: {f1_score(y_valid, pred[valid_idx]):.6f}')
    
print(f'\tF1 score: {f1_score(y, pred):.6f}')

##### 1. 현재 모델 성능 보기 
- (1) 제출 : 0.697899159
- (2) valid: 0.708

##### 2. K-fold 앙상블 적용하지 않고, 모델 생성 후 점수 보기
##### 3. fold 개수 변경해보기 
- (1) k=15 => 제출: 0.69691
- (2) k=20 =>	

##### 4. 피처 인포턴스 기반으로 피처 개수 정리
- 5개 정도 피처 제거 해보고 나머지 다른거

##### 5. one-hot-max 파라미터 수정
(1) 
- one_hot_max_size = 20
- k=7
- optuna 최적 하이퍼 파라미터 적용
(2)
- one_hot_max_size =10
- k=7
- optuna 최적 하이퍼 파라미터 적용

##### 6. 결정트리 스태킹
##### 7. 하이퍼 파라미터 적용
- optuna 최적 하이퍼 파라미터 적용 
- valid :0.71 제출: 0.69843


#Model 3 one_hot_max_size = 20

In [ ]:
#k 개의  모델을 다 사용하되 앙상블 처럼 각 모델에서 주는 output 들의 평균값을 사용하여 최종 pred 산출(OOF Ensemble)

skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = 0
cat_cols = X_train.columns[X_train.nunique() > 2].tolist()
for fold in range(n_fold):
  print(f'\n----------------- Fold {fold} -----------------\n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

  model_cat = CatBoostClassifier(max_depth=10,learning_rate=0.02,n_estimators=2099,max_bin=231,min_data_in_leaf=61,l2_leaf_reg=0.2193,subsample=0.6525,eval_metric="F1", task_type='GPU',bootstrap_type='Poisson', one_hot_max_size=20)
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
  
  cat_pred[valid_idx] = pd.DataFrame(model_cat.predict_proba(X_valid)[:, 1], index=X_valid.index)
  cat_pred_test = cat_pred_test + (model_cat.predict_proba(X_test)[:, 1] / n_fold)
  pred = np.where(cat_pred >= 0.4 , 1, 0)
  pred_test = np.where(cat_pred_test >= 0.4 , 1, 0)

  print(f'CV F1 score: {f1_score(y_valid, pred[valid_idx]):.6f}')
    
print(f'\tF1 score: {f1_score(y, pred):.6f}')



```
----------------- Fold 0 -----------------

0:	learn: 0.6251861	test: 0.6263958	best: 0.6263958 (0)	total: 698ms	remaining: 24m 25s
100:	learn: 0.6475928	test: 0.6477361	best: 0.6477361 (100)	total: 57s	remaining: 18m 46s
200:	learn: 0.6596423	test: 0.6606113	best: 0.6608103 (198)	total: 1m 53s	remaining: 17m 50s
300:	learn: 0.6691474	test: 0.6678405	best: 0.6679603 (295)	total: 2m 50s	remaining: 16m 55s
400:	learn: 0.6771361	test: 0.6706509	best: 0.6707179 (375)	total: 3m 45s	remaining: 15m 55s
500:	learn: 0.6841943	test: 0.6723041	best: 0.6723131 (499)	total: 4m 41s	remaining: 14m 57s
600:	learn: 0.6909437	test: 0.6734091	best: 0.6737398 (593)	total: 5m 34s	remaining: 13m 53s
700:	learn: 0.6973506	test: 0.6741371	best: 0.6742679 (686)	total: 6m 26s	remaining: 12m 51s
800:	learn: 0.7036685	test: 0.6740731	best: 0.6747530 (730)	total: 7m 19s	remaining: 11m 51s
bestTest = 0.6747529795
bestIteration = 730
Shrink model to first 731 iterations.
CV F1 score: 0.705351

----------------- Fold 1 -----------------

0:	learn: 0.6319002	test: 0.6334199	best: 0.6334199 (0)	total: 607ms	remaining: 21m 14s
100:	learn: 0.6485649	test: 0.6512819	best: 0.6512998 (98)	total: 56.5s	remaining: 18m 37s
200:	learn: 0.6607434	test: 0.6639592	best: 0.6639858 (198)	total: 1m 52s	remaining: 17m 39s
300:	learn: 0.6692984	test: 0.6679811	best: 0.6683066 (298)	total: 2m 48s	remaining: 16m 44s
400:	learn: 0.6778644	test: 0.6726720	best: 0.6728206 (397)	total: 3m 42s	remaining: 15m 41s
500:	learn: 0.6850085	test: 0.6740020	best: 0.6741150 (497)	total: 4m 39s	remaining: 14m 50s
600:	learn: 0.6916299	test: 0.6750486	best: 0.6752041 (593)	total: 5m 33s	remaining: 13m 52s
700:	learn: 0.6980000	test: 0.6766882	best: 0.6767342 (688)	total: 6m 27s	remaining: 12m 53s
800:	learn: 0.7038875	test: 0.6771395	best: 0.6772380 (795)	total: 7m 20s	remaining: 11m 54s
900:	learn: 0.7099811	test: 0.6775234	best: 0.6777386 (895)	total: 8m 13s	remaining: 10m 56s
1000:	learn: 0.7153245	test: 0.6779229	best: 0.6782558 (959)	total: 9m 4s	remaining: 9m 56s
1100:	learn: 0.7201424	test: 0.6782979	best: 0.6788506 (1063)	total: 9m 55s	remaining: 8m 59s
bestTest = 0.6788505503
bestIteration = 1063
Shrink model to first 1064 iterations.
CV F1 score: 0.707609

----------------- Fold 2 -----------------

0:	learn: 0.6326581	test: 0.6266336	best: 0.6266336 (0)	total: 695ms	remaining: 24m 17s
100:	learn: 0.6490684	test: 0.6433576	best: 0.6434332 (99)	total: 55.8s	remaining: 18m 23s
200:	learn: 0.6606139	test: 0.6548769	best: 0.6548815 (194)	total: 1m 52s	remaining: 17m 40s
300:	learn: 0.6706686	test: 0.6621457	best: 0.6621457 (300)	total: 2m 48s	remaining: 16m 45s
400:	learn: 0.6791609	test: 0.6647418	best: 0.6647418 (400)	total: 3m 43s	remaining: 15m 47s
500:	learn: 0.6867236	test: 0.6676882	best: 0.6677142 (499)	total: 4m 38s	remaining: 14m 48s
600:	learn: 0.6929215	test: 0.6697923	best: 0.6698804 (594)	total: 5m 31s	remaining: 13m 46s
700:	learn: 0.6992501	test: 0.6695477	best: 0.6704390 (619)	total: 6m 22s	remaining: 12m 43s
bestTest = 0.6704390347
bestIteration = 619
Shrink model to first 620 iterations.
CV F1 score: 0.702465

----------------- Fold 3 -----------------

0:	learn: 0.6255184	test: 0.6238619	best: 0.6238619 (0)	total: 723ms	remaining: 25m 17s
100:	learn: 0.6485452	test: 0.6506126	best: 0.6508597 (95)	total: 56.2s	remaining: 18m 31s
200:	learn: 0.6604876	test: 0.6640951	best: 0.6641641 (199)	total: 1m 52s	remaining: 17m 45s
300:	learn: 0.6701569	test: 0.6696937	best: 0.6696937 (300)	total: 2m 48s	remaining: 16m 47s
400:	learn: 0.6782281	test: 0.6715659	best: 0.6715659 (400)	total: 3m 44s	remaining: 15m 48s
500:	learn: 0.6853787	test: 0.6732039	best: 0.6733233 (499)	total: 4m 39s	remaining: 14m 50s
600:	learn: 0.6915586	test: 0.6749101	best: 0.6750558 (599)	total: 5m 32s	remaining: 13m 49s
700:	learn: 0.6984827	test: 0.6757107	best: 0.6757107 (700)	total: 6m 24s	remaining: 12m 47s
800:	learn: 0.7048791	test: 0.6771579	best: 0.6773301 (794)	total: 7m 14s	remaining: 11m 44s
900:	learn: 0.7104119	test: 0.6768007	best: 0.6775942 (831)	total: 8m 6s	remaining: 10m 47s
bestTest = 0.6775941872
bestIteration = 831
Shrink model to first 832 iterations.
CV F1 score: 0.706319

----------------- Fold 4 -----------------

0:	learn: 0.6245210	test: 0.6275553	best: 0.6275553 (0)	total: 614ms	remaining: 21m 27s
100:	learn: 0.6464366	test: 0.6471011	best: 0.6472435 (99)	total: 55.7s	remaining: 18m 21s
200:	learn: 0.6588890	test: 0.6612023	best: 0.6612023 (200)	total: 1m 51s	remaining: 17m 35s
300:	learn: 0.6689588	test: 0.6686738	best: 0.6686738 (299)	total: 2m 46s	remaining: 16m 36s
400:	learn: 0.6776242	test: 0.6710346	best: 0.6711548 (398)	total: 3m 43s	remaining: 15m 46s
500:	learn: 0.6848524	test: 0.6724523	best: 0.6725723 (476)	total: 4m 38s	remaining: 14m 48s
600:	learn: 0.6922577	test: 0.6745712	best: 0.6747007 (595)	total: 5m 32s	remaining: 13m 47s
700:	learn: 0.6989838	test: 0.6752917	best: 0.6753682 (695)	total: 6m 25s	remaining: 12m 48s
800:	learn: 0.7053536	test: 0.6762103	best: 0.6762103 (800)	total: 7m 18s	remaining: 11m 50s
900:	learn: 0.7105887	test: 0.6771964	best: 0.6774044 (893)	total: 8m 10s	remaining: 10m 51s
1000:	learn: 0.7158363	test: 0.6770096	best: 0.6775538 (926)	total: 9m 1s	remaining: 9m 53s
bestTest = 0.6775538293
bestIteration = 926
Shrink model to first 927 iterations.
CV F1 score: 0.707595

----------------- Fold 5 -----------------

0:	learn: 0.6198051	test: 0.6262259	best: 0.6262259 (0)	total: 662ms	remaining: 23m 7s
100:	learn: 0.6478593	test: 0.6447282	best: 0.6449458 (98)	total: 57.1s	remaining: 18m 48s
200:	learn: 0.6602349	test: 0.6591299	best: 0.6594166 (197)	total: 1m 52s	remaining: 17m 38s
300:	learn: 0.6695311	test: 0.6661682	best: 0.6663660 (296)	total: 2m 49s	remaining: 16m 49s
400:	learn: 0.6774738	test: 0.6685936	best: 0.6686973 (393)	total: 3m 44s	remaining: 15m 50s
500:	learn: 0.6850610	test: 0.6709758	best: 0.6710264 (496)	total: 4m 38s	remaining: 14m 47s
600:	learn: 0.6922163	test: 0.6724063	best: 0.6724063 (600)	total: 5m 31s	remaining: 13m 46s
700:	learn: 0.6985033	test: 0.6739953	best: 0.6739953 (700)	total: 6m 24s	remaining: 12m 45s
800:	learn: 0.7044885	test: 0.6752068	best: 0.6755404 (772)	total: 7m 16s	remaining: 11m 47s
bestTest = 0.6755404138
bestIteration = 772
Shrink model to first 773 iterations.
CV F1 score: 0.705002

----------------- Fold 6 -----------------

0:	learn: 0.6293640	test: 0.6361385	best: 0.6361385 (0)	total: 672ms	remaining: 23m 29s
100:	learn: 0.6479754	test: 0.6540539	best: 0.6543276 (93)	total: 56s	remaining: 18m 28s
200:	learn: 0.6583306	test: 0.6655087	best: 0.6656023 (199)	total: 1m 52s	remaining: 17m 42s
300:	learn: 0.6682678	test: 0.6714277	best: 0.6714949 (298)	total: 2m 48s	remaining: 16m 48s
400:	learn: 0.6769656	test: 0.6749085	best: 0.6749085 (400)	total: 3m 43s	remaining: 15m 46s
500:	learn: 0.6846385	test: 0.6770406	best: 0.6770406 (500)	total: 4m 37s	remaining: 14m 43s
600:	learn: 0.6914008	test: 0.6786257	best: 0.6786257 (600)	total: 5m 31s	remaining: 13m 46s
700:	learn: 0.6976550	test: 0.6798427	best: 0.6798890 (697)	total: 6m 24s	remaining: 12m 47s
800:	learn: 0.7041230	test: 0.6798000	best: 0.6799068 (703)	total: 7m 17s	remaining: 11m 48s
900:	learn: 0.7096360	test: 0.6797272	best: 0.6802553 (881)	total: 8m 8s	remaining: 10m 49s
1000:	learn: 0.7148863	test: 0.6798017	best: 0.6803971 (935)	total: 9m	remaining: 9m 53s
bestTest = 0.6803971467
bestIteration = 935
Shrink model to first 936 iterations.
CV F1 score: 0.708821
	F1 score: 0.706159
```



# Model 4 one_hot_max_size =10 / drop columns


In [17]:
drop_columns_serise=pd.Series(model_cat.feature_importances_,X_train.columns).sort_values(ascending=False)
drop_columns=drop_columns_serise[drop_columns_serise<0.08].index.tolist()


In [18]:
drop_columns

['tar_enc_1_10_div_person_prefer_d_1_s',
 'tar_enc_2_10_div_person_prefer_h_3_l',
 'tar_enc_2_10_div_person_prefer_d_3_s',
 'tar_enc_1_10_div_person_prefer_h_2',
 'tar_enc_1_10_div_person_prefer_e',
 'tar_enc_2_10_div_contents_attribute_e',
 'tar_enc_1_10_div_contents_attribute_l_n',
 'tar_enc_1_10_div_person_prefer_h_3_l',
 'tar_enc_2_10_div_person_prefer_d_3_l',
 'tar_enc_2_10_div_person_prefer_h_2',
 'tar_enc_2_10_div_person_prefer_d_2_s',
 'tar_enc_2_10_div_contents_attribute_l_s',
 'tar_enc_1_10_div_person_prefer_d_2_s',
 'tar_enc_2_10_div_person_prefer_d_2_m',
 'tar_enc_1_10_div_contents_attribute_l',
 'tar_enc_1_10_div_person_prefer_h_2_m',
 'tar_enc_2_10_div_person_prefer_d_2_n',
 'tar_enc_1_10_div_person_prefer_d_3_n',
 'tar_enc_1_10_div_contents_attribute_d_n',
 'tar_enc_1_10_div_person_prefer_h_3',
 'tar_enc_1_10_div_contents_attribute_l_l',
 'tar_enc_2_10_div_contents_attribute_l',
 'tar_enc_1_10_div_person_prefer_d_1_l',
 'tar_enc_1_10_div_person_prefer_d_2_l',
 'tar_enc_1

In [32]:
X_train = X_train.drop(drop_columns,axis=1)
X_test = X_test.drop(drop_columns,axis=1)

In [33]:
seed = 42
n_fold = 7
n_class = 1

X = X_train.copy()
y = y_train

In [34]:
#k 개의  모델을 다 사용하되 앙상블 처럼 각 모델에서 주는 output 들의 평균값을 사용하여 최종 pred 산출(OOF Ensemble)

skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))
importances = pd.Series()
cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = 0
cat_cols = X_train.columns[X_train.nunique() > 2].tolist()
for fold in range(n_fold):
  print(f'\n----------------- Fold {fold} -----------------\n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

  model_cat = CatBoostClassifier(max_depth=10,learning_rate=0.02,n_estimators=2099,max_bin=231,min_data_in_leaf=61,l2_leaf_reg=0.2193,subsample=0.6525,eval_metric="F1", task_type='GPU',bootstrap_type='Poisson', one_hot_max_size=10)
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
  importances += pd.Series(model_cat.feature_importances_,X_train.columns).sort_values(ascending=False)
  cat_pred[valid_idx] = pd.DataFrame(model_cat.predict_proba(X_valid)[:, 1], index=X_valid.index)
  cat_pred_test = cat_pred_test + (model_cat.predict_proba(X_test)[:, 1] / n_fold)
  pred = np.where(cat_pred >= 0.4 , 1, 0)
  pred_test = np.where(cat_pred_test >= 0.4 , 1, 0)

  print(f'CV F1 score: {f1_score(y_valid, pred[valid_idx]):.6f}')
    
print(f'\tF1 score: {f1_score(y, pred):.6f}')


----------------- Fold 0 -----------------

0:	learn: 0.6313282	test: 0.6332719	best: 0.6332719 (0)	total: 450ms	remaining: 15m 43s
100:	learn: 0.6473637	test: 0.6473985	best: 0.6473985 (100)	total: 34.3s	remaining: 11m 18s
200:	learn: 0.6605919	test: 0.6592481	best: 0.6594681 (199)	total: 1m 7s	remaining: 10m 33s
300:	learn: 0.6707730	test: 0.6675816	best: 0.6675816 (300)	total: 1m 38s	remaining: 9m 50s
400:	learn: 0.6790079	test: 0.6704275	best: 0.6706758 (397)	total: 2m 8s	remaining: 9m 5s
500:	learn: 0.6860096	test: 0.6723648	best: 0.6727437 (467)	total: 2m 38s	remaining: 8m 26s
600:	learn: 0.6933252	test: 0.6751602	best: 0.6752967 (599)	total: 3m 8s	remaining: 7m 49s
700:	learn: 0.6999172	test: 0.6753508	best: 0.6759147 (654)	total: 3m 37s	remaining: 7m 14s
800:	learn: 0.7060501	test: 0.6763141	best: 0.6766546 (796)	total: 4m 6s	remaining: 6m 39s
900:	learn: 0.7117635	test: 0.6769923	best: 0.6770851 (897)	total: 4m 35s	remaining: 6m 5s
1000:	learn: 0.7169785	test: 0.6775224	best:

In [35]:
submission['target'] = pred_test
submission

,id,target
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
46399,46399,1
46400,46400,1
46401,46401,1
46402,46402,1


In [37]:
submission.to_csv(path + "prediction_0125_1.csv", index=False)



```

----------------- Fold 0 -----------------

0:	learn: 0.6313282	test: 0.6332719	best: 0.6332719 (0)	total: 450ms	remaining: 15m 43s
100:	learn: 0.6473637	test: 0.6473985	best: 0.6473985 (100)	total: 34.3s	remaining: 11m 18s
200:	learn: 0.6605919	test: 0.6592481	best: 0.6594681 (199)	total: 1m 7s	remaining: 10m 33s
300:	learn: 0.6707730	test: 0.6675816	best: 0.6675816 (300)	total: 1m 38s	remaining: 9m 50s
400:	learn: 0.6790079	test: 0.6704275	best: 0.6706758 (397)	total: 2m 8s	remaining: 9m 5s
500:	learn: 0.6860096	test: 0.6723648	best: 0.6727437 (467)	total: 2m 38s	remaining: 8m 26s
600:	learn: 0.6933252	test: 0.6751602	best: 0.6752967 (599)	total: 3m 8s	remaining: 7m 49s
700:	learn: 0.6999172	test: 0.6753508	best: 0.6759147 (654)	total: 3m 37s	remaining: 7m 14s
800:	learn: 0.7060501	test: 0.6763141	best: 0.6766546 (796)	total: 4m 6s	remaining: 6m 39s
900:	learn: 0.7117635	test: 0.6769923	best: 0.6770851 (897)	total: 4m 35s	remaining: 6m 5s
1000:	learn: 0.7169785	test: 0.6775224	best: 0.6777175 (987)	total: 5m 3s	remaining: 5m 33s
1100:	learn: 0.7220042	test: 0.6772693	best: 0.6777622 (1019)	total: 5m 33s	remaining: 5m 2s
bestTest = 0.6777621817
bestIteration = 1019
Shrink model to first 1020 iterations.
CV F1 score: 0.709925

----------------- Fold 1 -----------------

0:	learn: 0.6305211	test: 0.6356816	best: 0.6356816 (0)	total: 429ms	remaining: 14m 59s
100:	learn: 0.6481844	test: 0.6499884	best: 0.6502807 (99)	total: 34.1s	remaining: 11m 14s
200:	learn: 0.6606543	test: 0.6609008	best: 0.6610607 (194)	total: 1m 7s	remaining: 10m 35s
300:	learn: 0.6697610	test: 0.6684831	best: 0.6686794 (295)	total: 1m 38s	remaining: 9m 50s
400:	learn: 0.6787603	test: 0.6726463	best: 0.6726463 (400)	total: 2m 9s	remaining: 9m 7s
500:	learn: 0.6861851	test: 0.6749286	best: 0.6749347 (496)	total: 2m 39s	remaining: 8m 30s
600:	learn: 0.6933786	test: 0.6772438	best: 0.6773070 (586)	total: 3m 9s	remaining: 7m 52s
700:	learn: 0.6997355	test: 0.6787826	best: 0.6790753 (693)	total: 3m 38s	remaining: 7m 16s
bestTest = 0.6790752505
bestIteration = 693
Shrink model to first 694 iterations.
CV F1 score: 0.704865

----------------- Fold 2 -----------------

0:	learn: 0.6283659	test: 0.6206557	best: 0.6206557 (0)	total: 413ms	remaining: 14m 26s
100:	learn: 0.6508687	test: 0.6457939	best: 0.6457939 (100)	total: 34.5s	remaining: 11m 21s
200:	learn: 0.6623751	test: 0.6563372	best: 0.6563372 (200)	total: 1m 7s	remaining: 10m 33s
300:	learn: 0.6723104	test: 0.6630494	best: 0.6630494 (300)	total: 1m 38s	remaining: 9m 47s
400:	learn: 0.6804516	test: 0.6659393	best: 0.6662024 (388)	total: 2m 8s	remaining: 9m 4s
500:	learn: 0.6875263	test: 0.6688474	best: 0.6688474 (500)	total: 2m 38s	remaining: 8m 26s
600:	learn: 0.6944829	test: 0.6718479	best: 0.6719002 (597)	total: 3m 7s	remaining: 7m 48s
700:	learn: 0.7008194	test: 0.6726600	best: 0.6729686 (697)	total: 3m 36s	remaining: 7m 12s
800:	learn: 0.7069674	test: 0.6726637	best: 0.6730544 (726)	total: 4m 5s	remaining: 6m 37s
bestTest = 0.673054371
bestIteration = 726
Shrink model to first 727 iterations.
CV F1 score: 0.704679

----------------- Fold 3 -----------------

0:	learn: 0.6091884	test: 0.6058551	best: 0.6058551 (0)	total: 403ms	remaining: 14m 5s
100:	learn: 0.6475752	test: 0.6475441	best: 0.6475441 (100)	total: 34.2s	remaining: 11m 15s
200:	learn: 0.6606213	test: 0.6625743	best: 0.6625743 (200)	total: 1m 6s	remaining: 10m 26s
300:	learn: 0.6706221	test: 0.6689162	best: 0.6689162 (300)	total: 1m 37s	remaining: 9m 43s
400:	learn: 0.6792506	test: 0.6713164	best: 0.6713164 (400)	total: 2m 8s	remaining: 9m 2s
500:	learn: 0.6863835	test: 0.6743018	best: 0.6745016 (485)	total: 2m 38s	remaining: 8m 25s
600:	learn: 0.6935246	test: 0.6753572	best: 0.6755700 (593)	total: 3m 7s	remaining: 7m 48s
700:	learn: 0.7002505	test: 0.6754843	best: 0.6761457 (656)	total: 3m 36s	remaining: 7m 12s
800:	learn: 0.7065592	test: 0.6764503	best: 0.6768372 (780)	total: 4m 5s	remaining: 6m 38s
900:	learn: 0.7119176	test: 0.6770900	best: 0.6771561 (896)	total: 4m 34s	remaining: 6m 4s
1000:	learn: 0.7171633	test: 0.6776998	best: 0.6777591 (994)	total: 5m 1s	remaining: 5m 31s
1100:	learn: 0.7216817	test: 0.6779000	best: 0.6783089 (1062)	total: 5m 29s	remaining: 4m 59s
1200:	learn: 0.7264259	test: 0.6781183	best: 0.6783095 (1161)	total: 5m 58s	remaining: 4m 27s
bestTest = 0.6783095123
bestIteration = 1161
Shrink model to first 1162 iterations.
CV F1 score: 0.710253

----------------- Fold 4 -----------------

0:	learn: 0.6285405	test: 0.6293393	best: 0.6293393 (0)	total: 431ms	remaining: 15m 3s
100:	learn: 0.6477522	test: 0.6490134	best: 0.6490134 (100)	total: 34.2s	remaining: 11m 16s
200:	learn: 0.6597697	test: 0.6601511	best: 0.6602389 (199)	total: 1m 6s	remaining: 10m 28s
300:	learn: 0.6697467	test: 0.6676759	best: 0.6676759 (300)	total: 1m 37s	remaining: 9m 44s
400:	learn: 0.6782384	test: 0.6712661	best: 0.6712661 (400)	total: 2m 8s	remaining: 9m 3s
500:	learn: 0.6863943	test: 0.6738654	best: 0.6740448 (496)	total: 2m 38s	remaining: 8m 27s
600:	learn: 0.6936006	test: 0.6761620	best: 0.6761620 (600)	total: 3m 8s	remaining: 7m 50s
700:	learn: 0.7002726	test: 0.6765883	best: 0.6774215 (641)	total: 3m 37s	remaining: 7m 14s
bestTest = 0.6774214583
bestIteration = 641
Shrink model to first 642 iterations.
CV F1 score: 0.705642

----------------- Fold 5 -----------------

0:	learn: 0.6153782	test: 0.6184271	best: 0.6184271 (0)	total: 399ms	remaining: 13m 56s
100:	learn: 0.6465729	test: 0.6430484	best: 0.6431589 (95)	total: 34.4s	remaining: 11m 20s
200:	learn: 0.6601136	test: 0.6571775	best: 0.6575014 (194)	total: 1m 6s	remaining: 10m 29s
300:	learn: 0.6701723	test: 0.6657955	best: 0.6660986 (289)	total: 1m 37s	remaining: 9m 44s
400:	learn: 0.6783583	test: 0.6695894	best: 0.6698233 (399)	total: 2m 8s	remaining: 9m 5s
500:	learn: 0.6866654	test: 0.6712809	best: 0.6713651 (496)	total: 2m 38s	remaining: 8m 26s
600:	learn: 0.6932174	test: 0.6727330	best: 0.6727330 (600)	total: 3m 7s	remaining: 7m 48s
700:	learn: 0.6996151	test: 0.6739656	best: 0.6741104 (696)	total: 3m 36s	remaining: 7m 12s
800:	learn: 0.7059376	test: 0.6746123	best: 0.6748145 (792)	total: 4m 5s	remaining: 6m 38s
900:	learn: 0.7117570	test: 0.6756746	best: 0.6756973 (890)	total: 4m 34s	remaining: 6m 4s
1000:	learn: 0.7174263	test: 0.6763414	best: 0.6765088 (993)	total: 5m 2s	remaining: 5m 32s
1100:	learn: 0.7223077	test: 0.6766792	best: 0.6768468 (1098)	total: 5m 31s	remaining: 5m
1200:	learn: 0.7272679	test: 0.6769264	best: 0.6769850 (1118)	total: 6m	remaining: 4m 29s
1300:	learn: 0.7318739	test: 0.6766761	best: 0.6770956 (1202)	total: 6m 28s	remaining: 3m 58s
bestTest = 0.6770956316
bestIteration = 1202
Shrink model to first 1203 iterations.
CV F1 score: 0.709284

----------------- Fold 6 -----------------

0:	learn: 0.6228161	test: 0.6284507	best: 0.6284507 (0)	total: 411ms	remaining: 14m 21s
100:	learn: 0.6479749	test: 0.6533577	best: 0.6533577 (100)	total: 33.8s	remaining: 11m 8s
200:	learn: 0.6598779	test: 0.6641076	best: 0.6641076 (200)	total: 1m 6s	remaining: 10m 25s
300:	learn: 0.6703575	test: 0.6714368	best: 0.6716253 (299)	total: 1m 37s	remaining: 9m 40s
400:	learn: 0.6783722	test: 0.6754247	best: 0.6754247 (400)	total: 2m 7s	remaining: 9m
500:	learn: 0.6862196	test: 0.6766262	best: 0.6768881 (456)	total: 2m 38s	remaining: 8m 24s
600:	learn: 0.6928413	test: 0.6785175	best: 0.6785770 (599)	total: 3m 7s	remaining: 7m 47s
700:	learn: 0.6994446	test: 0.6781602	best: 0.6786260 (652)	total: 3m 36s	remaining: 7m 11s
800:	learn: 0.7055928	test: 0.6791175	best: 0.6792363 (764)	total: 4m 5s	remaining: 6m 38s
900:	learn: 0.7112074	test: 0.6787209	best: 0.6800253 (831)	total: 4m 34s	remaining: 6m 5s
bestTest = 0.6800253278
bestIteration = 831
Shrink model to first 832 iterations.
CV F1 score: 0.709003
	F1 score: 0.707643
```



# Model 5 loss_log, cv = 7,est = 2500

In [ ]:
n_est = 2500
seed = 42
n_fold = 7
n_class = 1

X = X_train.copy()
y = y_train

In [ ]:
#k 개의  모델을 다 사용하되 앙상블 처럼 각 모델에서 주는 output 들의 평균값을 사용하여 최종 pred 산출(OOF Ensemble)

skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))
importances = pd.Series()
cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = 0
cat_cols = X_train.columns[X_train.nunique() >= 2].tolist()
for fold in range(n_fold):
  print(f'\n----------------- Fold {fold} -----------------\n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

  model_cat = CatBoostClassifier(eval_metric="F1", task_type='GPU',n_estimators=n_est,loss_function= 'Logloss')
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
  
  importances += pd.Series(model_cat.feature_importances_,X_train.columns).sort_values(ascending=False)
  cat_pred[valid_idx] = pd.DataFrame(model_cat.predict_proba(X_valid)[:, 1], index=X_valid.index)
  cat_pred_test = cat_pred_test + (model_cat.predict_proba(X_test)[:, 1] / n_fold)
  pred = np.where(cat_pred >= 0.4 , 1, 0)
  pred_test = np.where(cat_pred_test >= 0.4 , 1, 0)

  print(f'CV F1 score: {f1_score(y_valid, pred[valid_idx]):.6f}')
    
print(f'\tF1 score: {f1_score(y, pred):.6f}')



```

----------------- Fold 0 -----------------

Custom logger is already specified. Specify more than one logger at same time is not thread safe.Learning rate set to 0.029101
0:	learn: 0.6027932	test: 0.5999866	best: 0.5999866 (0)	total: 818ms	remaining: 34m 4s
100:	learn: 0.6389774	test: 0.6485756	best: 0.6485756 (100)	total: 1m 2s	remaining: 24m 42s
200:	learn: 0.6486279	test: 0.6635527	best: 0.6635527 (200)	total: 2m	remaining: 23m 2s
300:	learn: 0.6536141	test: 0.6702542	best: 0.6702876 (292)	total: 2m 57s	remaining: 21m 37s
400:	learn: 0.6577742	test: 0.6714986	best: 0.6717641 (388)	total: 3m 53s	remaining: 20m 20s
500:	learn: 0.6615209	test: 0.6733650	best: 0.6734340 (498)	total: 4m 48s	remaining: 19m 11s
600:	learn: 0.6644400	test: 0.6740241	best: 0.6745975 (586)	total: 5m 43s	remaining: 18m 5s
700:	learn: 0.6664918	test: 0.6756380	best: 0.6760201 (683)	total: 6m 37s	remaining: 17m
bestTest = 0.6760201378
bestIteration = 683
Shrink model to first 684 iterations.
CV F1 score: 0.707808

----------------- Fold 1 -----------------

Learning rate set to 0.029101
0:	learn: 0.6128201	test: 0.6174224	best: 0.6174224 (0)	total: 735ms	remaining: 30m 37s
100:	learn: 0.6405461	test: 0.6541918	best: 0.6541918 (100)	total: 1m 3s	remaining: 24m 58s
200:	learn: 0.6493793	test: 0.6652294	best: 0.6652472 (198)	total: 2m 1s	remaining: 23m 14s
300:	learn: 0.6540146	test: 0.6726097	best: 0.6726097 (300)	total: 2m 57s	remaining: 21m 37s
400:	learn: 0.6585676	test: 0.6756638	best: 0.6756638 (400)	total: 3m 54s	remaining: 20m 25s
500:	learn: 0.6616419	test: 0.6788564	best: 0.6788745 (497)	total: 4m 49s	remaining: 19m 14s
600:	learn: 0.6653165	test: 0.6790751	best: 0.6796402 (529)	total: 5m 44s	remaining: 18m 7s
700:	learn: 0.6682468	test: 0.6777666	best: 0.6800105 (626)	total: 6m 38s	remaining: 17m 3s
bestTest = 0.6800105001
bestIteration = 626
Shrink model to first 627 iterations.
CV F1 score: 0.706628

----------------- Fold 2 -----------------

Learning rate set to 0.029101
0:	learn: 0.6128748	test: 0.6083786	best: 0.6083786 (0)	total: 736ms	remaining: 30m 38s
100:	learn: 0.6393916	test: 0.6418444	best: 0.6418717 (99)	total: 1m 1s	remaining: 24m 21s
200:	learn: 0.6489608	test: 0.6553756	best: 0.6557543 (196)	total: 1m 59s	remaining: 22m 50s
300:	learn: 0.6547551	test: 0.6626622	best: 0.6626622 (300)	total: 2m 56s	remaining: 21m 26s
400:	learn: 0.6592850	test: 0.6670859	best: 0.6673651 (395)	total: 3m 52s	remaining: 20m 17s
500:	learn: 0.6627849	test: 0.6697572	best: 0.6699380 (481)	total: 4m 47s	remaining: 19m 7s
600:	learn: 0.6664274	test: 0.6718565	best: 0.6718565 (600)	total: 5m 42s	remaining: 18m 3s
700:	learn: 0.6688910	test: 0.6739258	best: 0.6742233 (680)	total: 6m 37s	remaining: 16m 59s
800:	learn: 0.6708150	test: 0.6753024	best: 0.6753024 (800)	total: 7m 31s	remaining: 15m 57s
900:	learn: 0.6722748	test: 0.6760681	best: 0.6762110 (898)	total: 8m 24s	remaining: 14m 56s
1000:	learn: 0.6733599	test: 0.6775232	best: 0.6775232 (1000)	total: 9m 19s	remaining: 13m 57s
1100:	learn: 0.6744322	test: 0.6781293	best: 0.6781293 (1100)	total: 10m 12s	remaining: 12m 58s
1200:	learn: 0.6754288	test: 0.6782484	best: 0.6782484 (1200)	total: 11m 6s	remaining: 12m
1300:	learn: 0.6762860	test: 0.6784874	best: 0.6787915 (1293)	total: 11m 59s	remaining: 11m 3s
bestTest = 0.678791482
bestIteration = 1293
Shrink model to first 1294 iterations.
CV F1 score: 0.709444

----------------- Fold 3 -----------------

Learning rate set to 0.029101
0:	learn: 0.5799073	test: 0.5821077	best: 0.5821077 (0)	total: 822ms	remaining: 34m 13s
100:	learn: 0.6391378	test: 0.6502048	best: 0.6502048 (100)	total: 1m 2s	remaining: 24m 55s
200:	learn: 0.6484468	test: 0.6636190	best: 0.6636190 (200)	total: 2m 2s	remaining: 23m 16s
300:	learn: 0.6544803	test: 0.6689044	best: 0.6689304 (299)	total: 2m 57s	remaining: 21m 40s
400:	learn: 0.6586064	test: 0.6726529	best: 0.6726974 (391)	total: 3m 53s	remaining: 20m 24s
500:	learn: 0.6622080	test: 0.6744962	best: 0.6746025 (489)	total: 4m 48s	remaining: 19m 9s
600:	learn: 0.6654927	test: 0.6754688	best: 0.6761200 (589)	total: 5m 41s	remaining: 18m
700:	learn: 0.6676997	test: 0.6769390	best: 0.6771322 (691)	total: 6m 35s	remaining: 16m 55s
800:	learn: 0.6698317	test: 0.6765447	best: 0.6771952 (713)	total: 7m 30s	remaining: 15m 54s
bestTest = 0.6771951655
bestIteration = 713
Shrink model to first 714 iterations.
CV F1 score: 0.707998

----------------- Fold 4 -----------------

Learning rate set to 0.029101
0:	learn: 0.6063960	test: 0.6096207	best: 0.6096207 (0)	total: 721ms	remaining: 30m 1s
100:	learn: 0.6391960	test: 0.6489370	best: 0.6489370 (100)	total: 1m 1s	remaining: 24m 23s
200:	learn: 0.6481677	test: 0.6643200	best: 0.6643200 (200)	total: 2m	remaining: 22m 59s
300:	learn: 0.6538195	test: 0.6693863	best: 0.6693863 (300)	total: 2m 56s	remaining: 21m 32s
400:	learn: 0.6583477	test: 0.6723811	best: 0.6725143 (392)	total: 3m 52s	remaining: 20m 18s
500:	learn: 0.6625456	test: 0.6762193	best: 0.6763899 (497)	total: 4m 49s	remaining: 19m 14s
600:	learn: 0.6653854	test: 0.6773263	best: 0.6774532 (598)	total: 5m 43s	remaining: 18m 6s
700:	learn: 0.6681811	test: 0.6761971	best: 0.6775345 (603)	total: 6m 38s	remaining: 17m 3s
bestTest = 0.6775345199
bestIteration = 603
Shrink model to first 604 iterations.
CV F1 score: 0.706865

----------------- Fold 5 -----------------

Learning rate set to 0.029101
0:	learn: 0.6056353	test: 0.6081692	best: 0.6081692 (0)	total: 736ms	remaining: 30m 40s
100:	learn: 0.6396504	test: 0.6491199	best: 0.6491199 (100)	total: 1m 1s	remaining: 24m 32s
200:	learn: 0.6490388	test: 0.6631909	best: 0.6631909 (200)	total: 2m	remaining: 22m 53s
300:	learn: 0.6544709	test: 0.6694459	best: 0.6694459 (300)	total: 2m 57s	remaining: 21m 33s
400:	learn: 0.6585750	test: 0.6717123	best: 0.6717902 (398)	total: 3m 52s	remaining: 20m 18s
500:	learn: 0.6625779	test: 0.6742077	best: 0.6744405 (484)	total: 4m 48s	remaining: 19m 9s
600:	learn: 0.6655290	test: 0.6770189	best: 0.6770189 (600)	total: 5m 43s	remaining: 18m 5s
700:	learn: 0.6676688	test: 0.6775010	best: 0.6775096 (633)	total: 6m 39s	remaining: 17m 3s
bestTest = 0.6775096372
bestIteration = 633
Shrink model to first 634 iterations.
CV F1 score: 0.705286

----------------- Fold 6 -----------------

Learning rate set to 0.029101
0:	learn: 0.6327623	test: 0.6383275	best: 0.6383275 (0)	total: 825ms	remaining: 34m 22s
100:	learn: 0.6389214	test: 0.6543704	best: 0.6543704 (100)	total: 1m 1s	remaining: 24m 22s
200:	learn: 0.6485154	test: 0.6689118	best: 0.6689118 (200)	total: 1m 59s	remaining: 22m 44s
300:	learn: 0.6542150	test: 0.6742064	best: 0.6742064 (300)	total: 2m 55s	remaining: 21m 22s
400:	learn: 0.6579275	test: 0.6778875	best: 0.6779565 (398)	total: 3m 50s	remaining: 20m 7s
500:	learn: 0.6620937	test: 0.6794187	best: 0.6795396 (499)	total: 4m 45s	remaining: 18m 57s
600:	learn: 0.6648922	test: 0.6806789	best: 0.6811534 (584)	total: 5m 40s	remaining: 17m 54s
700:	learn: 0.6673297	test: 0.6820608	best: 0.6822423 (682)	total: 6m 34s	remaining: 16m 52s
800:	learn: 0.6697228	test: 0.6822959	best: 0.6824606 (759)	total: 7m 28s	remaining: 15m 50s
bestTest = 0.6824605919
bestIteration = 759
Shrink model to first 760 iterations.
CV F1 score: 0.710710
	F1 score: 0.707811
```



# MOdel 6 
- k=20

In [ ]:
n_est = 2500
seed = 42
n_fold = 20
n_class = 1

X = X_train.copy()
y = y_train

In [ ]:
#k 개의  모델을 다 사용하되 앙상블 처럼 각 모델에서 주는 output 들의 평균값을 사용하여 최종 pred 산출(OOF Ensemble)

skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))
importances = pd.Series()
cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = 0
cat_cols = X_train.columns[X_train.nunique() >= 2].tolist()
for fold in range(n_fold):
  print(f'\n----------------- Fold {fold} -----------------\n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

  model_cat = CatBoostClassifier(eval_metric="F1", task_type='GPU',n_estimators=n_est,loss_function= 'Logloss')
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
  
  importances += pd.Series(model_cat.feature_importances_,X_train.columns).sort_values(ascending=False)
  cat_pred[valid_idx] = pd.DataFrame(model_cat.predict_proba(X_valid)[:, 1], index=X_valid.index)
  cat_pred_test = cat_pred_test + (model_cat.predict_proba(X_test)[:, 1] / n_fold)
  pred = np.where(cat_pred >= 0.4 , 1, 0)
  pred_test = np.where(cat_pred_test >= 0.4 , 1, 0)

  print(f'CV F1 score: {f1_score(y_valid, pred[valid_idx]):.6f}')
    
print(f'\tF1 score: {f1_score(y, pred):.6f}')



```
----------------- Fold 0 -----------------

Learning rate set to 0.028848
0:	learn: 0.5702392	test: 0.5692207	best: 0.5692207 (0)	total: 934ms	remaining: 38m 54s
100:	learn: 0.6395766	test: 0.6488468	best: 0.6490390 (96)	total: 1m 16s	remaining: 30m 9s
200:	learn: 0.6503510	test: 0.6680949	best: 0.6680949 (200)	total: 2m 27s	remaining: 28m 9s
300:	learn: 0.6552536	test: 0.6749426	best: 0.6749426 (300)	total: 3m 36s	remaining: 26m 18s
400:	learn: 0.6597827	test: 0.6766839	best: 0.6772301 (399)	total: 4m 43s	remaining: 24m 42s
500:	learn: 0.6634491	test: 0.6794655	best: 0.6800464 (478)	total: 5m 50s	remaining: 23m 18s
600:	learn: 0.6665827	test: 0.6824117	best: 0.6824838 (596)	total: 6m 55s	remaining: 21m 54s
700:	learn: 0.6692622	test: 0.6824167	best: 0.6835048 (690)	total: 8m 3s	remaining: 20m 39s
bestTest = 0.6835047619
bestIteration = 690
Shrink model to first 691 iterations.
CV F1 score: 0.712314

----------------- Fold 1 -----------------

Learning rate set to 0.028848
0:	learn: 0.5855280	test: 0.5780824	best: 0.5780824 (0)	total: 945ms	remaining: 39m 21s
100:	learn: 0.6403387	test: 0.6466797	best: 0.6468163 (95)	total: 1m 14s	remaining: 29m 27s
200:	learn: 0.6501969	test: 0.6634753	best: 0.6637706 (198)	total: 2m 25s	remaining: 27m 39s
300:	learn: 0.6557676	test: 0.6691763	best: 0.6692983 (288)	total: 3m 33s	remaining: 26m 2s
400:	learn: 0.6602057	test: 0.6715790	best: 0.6718023 (398)	total: 4m 40s	remaining: 24m 28s
500:	learn: 0.6640143	test: 0.6731353	best: 0.6734268 (447)	total: 5m 47s	remaining: 23m 4s
bestTest = 0.673426782
bestIteration = 447
Shrink model to first 448 iterations.
CV F1 score: 0.703805

----------------- Fold 2 -----------------

Learning rate set to 0.028848
0:	learn: 0.6238606	test: 0.6233172	best: 0.6233172 (0)	total: 845ms	remaining: 35m 11s
100:	learn: 0.6399524	test: 0.6489287	best: 0.6489287 (100)	total: 1m 15s	remaining: 29m 49s
200:	learn: 0.6501867	test: 0.6644318	best: 0.6650214 (197)	total: 2m 28s	remaining: 28m 14s
300:	learn: 0.6555962	test: 0.6694617	best: 0.6694617 (300)	total: 3m 36s	remaining: 26m 19s
400:	learn: 0.6606899	test: 0.6720651	best: 0.6723109 (395)	total: 4m 42s	remaining: 24m 40s
500:	learn: 0.6639841	test: 0.6750384	best: 0.6758285 (494)	total: 5m 49s	remaining: 23m 15s
600:	learn: 0.6669850	test: 0.6765071	best: 0.6771169 (535)	total: 6m 55s	remaining: 21m 53s
bestTest = 0.6771169279
bestIteration = 535
Shrink model to first 536 iterations.
CV F1 score: 0.703376

----------------- Fold 3 -----------------

Learning rate set to 0.028848
0:	learn: 0.6038384	test: 0.6157482	best: 0.6157482 (0)	total: 856ms	remaining: 35m 39s
100:	learn: 0.6399068	test: 0.6542484	best: 0.6542484 (100)	total: 1m 14s	remaining: 29m 27s
200:	learn: 0.6508809	test: 0.6695643	best: 0.6695643 (200)	total: 2m 25s	remaining: 27m 43s
300:	learn: 0.6563671	test: 0.6751142	best: 0.6752146 (299)	total: 3m 32s	remaining: 25m 54s
400:	learn: 0.6608207	test: 0.6774517	best: 0.6775009 (399)	total: 4m 40s	remaining: 24m 27s
500:	learn: 0.6648965	test: 0.6790267	best: 0.6797130 (499)	total: 5m 49s	remaining: 23m 13s
600:	learn: 0.6677567	test: 0.6806437	best: 0.6807499 (588)	total: 6m 53s	remaining: 21m 46s
700:	learn: 0.6697625	test: 0.6831559	best: 0.6831559 (700)	total: 7m 59s	remaining: 20m 30s
800:	learn: 0.6715266	test: 0.6841045	best: 0.6842565 (799)	total: 9m 4s	remaining: 19m 15s
900:	learn: 0.6730428	test: 0.6837469	best: 0.6843872 (824)	total: 10m 9s	remaining: 18m 2s
1000:	learn: 0.6745681	test: 0.6846909	best: 0.6850816 (952)	total: 11m 14s	remaining: 16m 49s
1100:	learn: 0.6759619	test: 0.6851717	best: 0.6858307 (1081)	total: 12m 18s	remaining: 15m 38s
bestTest = 0.6858306752
bestIteration = 1081
Shrink model to first 1082 iterations.
CV F1 score: 0.713815

----------------- Fold 4 -----------------

Learning rate set to 0.028848
0:	learn: 0.6137439	test: 0.6182854	best: 0.6182854 (0)	total: 862ms	remaining: 35m 53s
100:	learn: 0.6388906	test: 0.6458972	best: 0.6461029 (85)	total: 1m 15s	remaining: 29m 42s
200:	learn: 0.6487639	test: 0.6584133	best: 0.6586363 (199)	total: 2m 24s	remaining: 27m 34s
300:	learn: 0.6553731	test: 0.6675609	best: 0.6681569 (298)	total: 3m 33s	remaining: 25m 57s
400:	learn: 0.6595660	test: 0.6724711	best: 0.6729195 (396)	total: 4m 39s	remaining: 24m 24s
500:	learn: 0.6633875	test: 0.6767783	best: 0.6767783 (500)	total: 5m 46s	remaining: 23m 2s
600:	learn: 0.6663444	test: 0.6767899	best: 0.6771367 (582)	total: 6m 52s	remaining: 21m 44s
700:	learn: 0.6689291	test: 0.6780477	best: 0.6780477 (700)	total: 7m 58s	remaining: 20m 28s
800:	learn: 0.6704208	test: 0.6794533	best: 0.6797045 (796)	total: 9m 3s	remaining: 19m 13s
900:	learn: 0.6721836	test: 0.6817606	best: 0.6817606 (900)	total: 10m 9s	remaining: 18m 1s
1000:	learn: 0.6735246	test: 0.6818059	best: 0.6823149 (998)	total: 11m 14s	remaining: 16m 50s
1100:	learn: 0.6746040	test: 0.6821514	best: 0.6830546 (1075)	total: 12m 19s	remaining: 15m 40s
bestTest = 0.6830546351
bestIteration = 1075
Shrink model to first 1076 iterations.
CV F1 score: 0.713222

----------------- Fold 5 -----------------

Learning rate set to 0.028848
0:	learn: 0.6105431	test: 0.6100348	best: 0.6100348 (0)	total: 838ms	remaining: 34m 55s
100:	learn: 0.6396562	test: 0.6491597	best: 0.6497398 (98)	total: 1m 15s	remaining: 29m 56s
200:	learn: 0.6491666	test: 0.6633685	best: 0.6635670 (198)	total: 2m 25s	remaining: 27m 49s
300:	learn: 0.6547997	test: 0.6693149	best: 0.6695623 (289)	total: 3m 34s	remaining: 26m 6s
400:	learn: 0.6592932	test: 0.6740535	best: 0.6740788 (398)	total: 4m 40s	remaining: 24m 29s
500:	learn: 0.6628414	test: 0.6773464	best: 0.6775495 (496)	total: 5m 46s	remaining: 23m 4s
600:	learn: 0.6668094	test: 0.6813187	best: 0.6813187 (600)	total: 6m 52s	remaining: 21m 43s
700:	learn: 0.6697277	test: 0.6828189	best: 0.6835366 (674)	total: 7m 58s	remaining: 20m 28s
bestTest = 0.6835365854
bestIteration = 674
Shrink model to first 675 iterations.
CV F1 score: 0.711158

----------------- Fold 6 -----------------

Learning rate set to 0.028848
0:	learn: 0.5708947	test: 0.5686893	best: 0.5686893 (0)	total: 955ms	remaining: 39m 47s
100:	learn: 0.6406582	test: 0.6474098	best: 0.6474573 (99)	total: 1m 15s	remaining: 30m 4s
200:	learn: 0.6500146	test: 0.6638058	best: 0.6640241 (192)	total: 2m 27s	remaining: 28m 7s
300:	learn: 0.6556981	test: 0.6683959	best: 0.6688117 (294)	total: 3m 35s	remaining: 26m 15s
400:	learn: 0.6596780	test: 0.6709519	best: 0.6712476 (389)	total: 4m 41s	remaining: 24m 33s
500:	learn: 0.6642177	test: 0.6748857	best: 0.6748857 (500)	total: 5m 48s	remaining: 23m 9s
600:	learn: 0.6672026	test: 0.6772618	best: 0.6774872 (596)	total: 6m 55s	remaining: 21m 51s
700:	learn: 0.6697015	test: 0.6778995	best: 0.6783705 (695)	total: 8m	remaining: 20m 33s
800:	learn: 0.6716127	test: 0.6791016	best: 0.6791016 (800)	total: 9m 7s	remaining: 19m 20s
900:	learn: 0.6734443	test: 0.6800703	best: 0.6802019 (899)	total: 10m 11s	remaining: 18m 5s
1000:	learn: 0.6746556	test: 0.6803530	best: 0.6807828 (919)	total: 11m 16s	remaining: 16m 53s
bestTest = 0.6807827549
bestIteration = 919
Shrink model to first 920 iterations.
CV F1 score: 0.711416

----------------- Fold 7 -----------------

Learning rate set to 0.028848
0:	learn: 0.5687612	test: 0.5622782	best: 0.5622782 (0)	total: 956ms	remaining: 39m 49s
100:	learn: 0.6402481	test: 0.6476177	best: 0.6476177 (100)	total: 1m 15s	remaining: 29m 56s
200:	learn: 0.6502934	test: 0.6604710	best: 0.6604947 (198)	total: 2m 26s	remaining: 27m 59s
300:	learn: 0.6566554	test: 0.6665174	best: 0.6671377 (288)	total: 3m 34s	remaining: 26m 9s
400:	learn: 0.6611173	test: 0.6705194	best: 0.6708191 (398)	total: 4m 42s	remaining: 24m 36s
500:	learn: 0.6646695	test: 0.6729965	best: 0.6729965 (500)	total: 5m 49s	remaining: 23m 13s
600:	learn: 0.6675150	test: 0.6760649	best: 0.6760649 (600)	total: 6m 54s	remaining: 21m 48s
700:	learn: 0.6697700	test: 0.6773009	best: 0.6783626 (640)	total: 8m 1s	remaining: 20m 35s
800:	learn: 0.6717047	test: 0.6779022	best: 0.6793814 (747)	total: 9m 6s	remaining: 19m 19s
bestTest = 0.6793814037
bestIteration = 747
Shrink model to first 748 iterations.
CV F1 score: 0.709443

----------------- Fold 8 -----------------

Learning rate set to 0.028848
0:	learn: 0.6171077	test: 0.6226756	best: 0.6226756 (0)	total: 845ms	remaining: 35m 12s
100:	learn: 0.6390721	test: 0.6490144	best: 0.6490144 (100)	total: 1m 16s	remaining: 30m 9s
200:	learn: 0.6504492	test: 0.6673254	best: 0.6673254 (200)	total: 2m 27s	remaining: 28m 4s
300:	learn: 0.6565073	test: 0.6728999	best: 0.6729226 (298)	total: 3m 35s	remaining: 26m 17s
400:	learn: 0.6605336	test: 0.6764217	best: 0.6769140 (394)	total: 4m 43s	remaining: 24m 41s
500:	learn: 0.6644076	test: 0.6781093	best: 0.6782323 (499)	total: 5m 49s	remaining: 23m 15s
600:	learn: 0.6679029	test: 0.6802172	best: 0.6803961 (554)	total: 6m 56s	remaining: 21m 56s
700:	learn: 0.6701204	test: 0.6799803	best: 0.6806137 (604)	total: 8m 2s	remaining: 20m 39s
bestTest = 0.6806136681
bestIteration = 604
Shrink model to first 605 iterations.
CV F1 score: 0.707410

----------------- Fold 9 -----------------

Learning rate set to 0.028848
0:	learn: 0.5932809	test: 0.5872910	best: 0.5872910 (0)	total: 943ms	remaining: 39m 15s
100:	learn: 0.6412302	test: 0.6505043	best: 0.6505043 (100)	total: 1m 16s	remaining: 30m 23s
200:	learn: 0.6509027	test: 0.6644128	best: 0.6647074 (199)	total: 2m 27s	remaining: 28m 10s
300:	learn: 0.6564293	test: 0.6705587	best: 0.6707290 (291)	total: 3m 35s	remaining: 26m 13s
400:	learn: 0.6608932	test: 0.6726270	best: 0.6731213 (397)	total: 4m 41s	remaining: 24m 36s
500:	learn: 0.6648497	test: 0.6745775	best: 0.6751530 (496)	total: 5m 48s	remaining: 23m 11s
600:	learn: 0.6675155	test: 0.6749744	best: 0.6762649 (537)	total: 6m 55s	remaining: 21m 53s
bestTest = 0.6762649425
bestIteration = 537
Shrink model to first 538 iterations.
CV F1 score: 0.706710

----------------- Fold 10 -----------------

Learning rate set to 0.028848
0:	learn: 0.6158146	test: 0.6135684	best: 0.6135684 (0)	total: 862ms	remaining: 35m 53s
100:	learn: 0.6393571	test: 0.6480154	best: 0.6483910 (98)	total: 1m 15s	remaining: 29m 59s
200:	learn: 0.6498218	test: 0.6607308	best: 0.6617288 (180)	total: 2m 27s	remaining: 28m 8s
300:	learn: 0.6560577	test: 0.6686498	best: 0.6686498 (300)	total: 3m 35s	remaining: 26m 12s
400:	learn: 0.6604100	test: 0.6741514	best: 0.6744221 (389)	total: 4m 42s	remaining: 24m 36s
500:	learn: 0.6640533	test: 0.6769892	best: 0.6770598 (497)	total: 5m 49s	remaining: 23m 14s
600:	learn: 0.6676524	test: 0.6798056	best: 0.6801214 (594)	total: 6m 56s	remaining: 21m 55s
700:	learn: 0.6698826	test: 0.6802011	best: 0.6812002 (656)	total: 8m 2s	remaining: 20m 39s
800:	learn: 0.6713802	test: 0.6817748	best: 0.6821250 (768)	total: 9m 9s	remaining: 19m 24s
900:	learn: 0.6728435	test: 0.6824503	best: 0.6830072 (852)	total: 10m 13s	remaining: 18m 9s
1000:	learn: 0.6742037	test: 0.6834637	best: 0.6837463 (994)	total: 11m 19s	remaining: 16m 57s
1100:	learn: 0.6749787	test: 0.6838690	best: 0.6840206 (1097)	total: 12m 24s	remaining: 15m 46s
1200:	learn: 0.6760887	test: 0.6841497	best: 0.6841943 (1148)	total: 13m 29s	remaining: 14m 35s
1300:	learn: 0.6767954	test: 0.6835864	best: 0.6842855 (1225)	total: 14m 34s	remaining: 13m 25s
1400:	learn: 0.6776742	test: 0.6843957	best: 0.6848045 (1386)	total: 15m 38s	remaining: 12m 16s
1500:	learn: 0.6783411	test: 0.6848372	best: 0.6850385 (1492)	total: 16m 43s	remaining: 11m 8s
1600:	learn: 0.6790284	test: 0.6843612	best: 0.6850406 (1518)	total: 17m 48s	remaining: 10m
bestTest = 0.6850405876
bestIteration = 1518
Shrink model to first 1519 iterations.
CV F1 score: 0.714682

----------------- Fold 11 -----------------

Learning rate set to 0.028848
0:	learn: 0.5698026	test: 0.5615569	best: 0.5615569 (0)	total: 965ms	remaining: 40m 11s
100:	learn: 0.6407615	test: 0.6399705	best: 0.6399705 (100)	total: 1m 14s	remaining: 29m 35s
200:	learn: 0.6511509	test: 0.6619878	best: 0.6619878 (200)	total: 2m 25s	remaining: 27m 40s
300:	learn: 0.6566728	test: 0.6658542	best: 0.6663719 (296)	total: 3m 31s	remaining: 25m 46s
400:	learn: 0.6604215	test: 0.6704276	best: 0.6704276 (400)	total: 4m 40s	remaining: 24m 26s
500:	learn: 0.6643394	test: 0.6734381	best: 0.6737721 (482)	total: 5m 46s	remaining: 23m 4s
600:	learn: 0.6674611	test: 0.6743263	best: 0.6747910 (558)	total: 6m 53s	remaining: 21m 46s
700:	learn: 0.6699315	test: 0.6755309	best: 0.6760531 (684)	total: 7m 58s	remaining: 20m 28s
800:	learn: 0.6715623	test: 0.6761359	best: 0.6762426 (713)	total: 9m 5s	remaining: 19m 17s
bestTest = 0.6762425901
bestIteration = 713
Shrink model to first 714 iterations.
CV F1 score: 0.710833

----------------- Fold 12 -----------------

Learning rate set to 0.028848
0:	learn: 0.6195568	test: 0.6221340	best: 0.6221340 (0)	total: 872ms	remaining: 36m 17s
100:	learn: 0.6405168	test: 0.6517464	best: 0.6520654 (99)	total: 1m 15s	remaining: 29m 56s
200:	learn: 0.6502604	test: 0.6678171	best: 0.6678171 (200)	total: 2m 26s	remaining: 27m 52s
300:	learn: 0.6559710	test: 0.6738801	best: 0.6738801 (300)	total: 3m 34s	remaining: 26m 6s
400:	learn: 0.6606496	test: 0.6794929	best: 0.6796930 (399)	total: 4m 41s	remaining: 24m 33s
500:	learn: 0.6649308	test: 0.6816227	best: 0.6818471 (496)	total: 5m 48s	remaining: 23m 10s
600:	learn: 0.6676542	test: 0.6822010	best: 0.6826600 (596)	total: 6m 54s	remaining: 21m 50s
700:	learn: 0.6698365	test: 0.6832492	best: 0.6833371 (659)	total: 8m	remaining: 20m 32s
800:	learn: 0.6716795	test: 0.6835462	best: 0.6841463 (772)	total: 9m 5s	remaining: 19m 18s
900:	learn: 0.6733226	test: 0.6841482	best: 0.6845858 (822)	total: 10m 10s	remaining: 18m 3s
bestTest = 0.6845857638
bestIteration = 822
Shrink model to first 823 iterations.
CV F1 score: 0.713944

----------------- Fold 13 -----------------

Learning rate set to 0.028848
0:	learn: 0.6392666	test: 0.6406320	best: 0.6406320 (0)	total: 978ms	remaining: 40m 42s
100:	learn: 0.6407015	test: 0.6502221	best: 0.6502452 (88)	total: 1m 15s	remaining: 29m 50s
200:	learn: 0.6504128	test: 0.6652986	best: 0.6652986 (200)	total: 2m 26s	remaining: 27m 57s
300:	learn: 0.6557757	test: 0.6711919	best: 0.6711919 (300)	total: 3m 35s	remaining: 26m 12s
400:	learn: 0.6602540	test: 0.6742506	best: 0.6744875 (390)	total: 4m 42s	remaining: 24m 38s
500:	learn: 0.6635855	test: 0.6750989	best: 0.6762740 (457)	total: 5m 49s	remaining: 23m 12s
600:	learn: 0.6673297	test: 0.6777803	best: 0.6779852 (573)	total: 6m 54s	remaining: 21m 51s
700:	learn: 0.6693780	test: 0.6792766	best: 0.6794403 (695)	total: 8m 1s	remaining: 20m 34s
800:	learn: 0.6715550	test: 0.6808592	best: 0.6811033 (793)	total: 9m 7s	remaining: 19m 21s
900:	learn: 0.6730634	test: 0.6813892	best: 0.6816914 (893)	total: 10m 12s	remaining: 18m 6s
1000:	learn: 0.6743386	test: 0.6828074	best: 0.6828467 (950)	total: 11m 16s	remaining: 16m 53s
1100:	learn: 0.6753345	test: 0.6820729	best: 0.6828578 (1002)	total: 12m 21s	remaining: 15m 42s
bestTest = 0.6828577985
bestIteration = 1002
Shrink model to first 1003 iterations.
CV F1 score: 0.711344

----------------- Fold 14 -----------------

Learning rate set to 0.028848
0:	learn: 0.6126379	test: 0.6129262	best: 0.6129262 (0)	total: 838ms	remaining: 34m 55s
100:	learn: 0.6404467	test: 0.6417285	best: 0.6417285 (100)	total: 1m 16s	remaining: 30m 19s
200:	learn: 0.6497819	test: 0.6534808	best: 0.6534808 (200)	total: 2m 27s	remaining: 28m 8s
300:	learn: 0.6555326	test: 0.6626081	best: 0.6627065 (297)	total: 3m 35s	remaining: 26m 17s
400:	learn: 0.6602109	test: 0.6657387	best: 0.6663670 (385)	total: 4m 42s	remaining: 24m 39s
500:	learn: 0.6644821	test: 0.6699135	best: 0.6699891 (495)	total: 5m 49s	remaining: 23m 14s
600:	learn: 0.6671347	test: 0.6726039	best: 0.6728547 (598)	total: 6m 56s	remaining: 21m 55s
700:	learn: 0.6696516	test: 0.6741060	best: 0.6744354 (697)	total: 8m 3s	remaining: 20m 39s
800:	learn: 0.6714792	test: 0.6732469	best: 0.6745499 (744)	total: 9m 8s	remaining: 19m 23s
bestTest = 0.6745498932
bestIteration = 744
Shrink model to first 745 iterations.
CV F1 score: 0.705560

----------------- Fold 15 -----------------

Learning rate set to 0.028848
0:	learn: 0.5727654	test: 0.5719760	best: 0.5719760 (0)	total: 969ms	remaining: 40m 20s
100:	learn: 0.6396605	test: 0.6508383	best: 0.6512154 (98)	total: 1m 15s	remaining: 29m 54s
200:	learn: 0.6502187	test: 0.6660821	best: 0.6661309 (199)	total: 2m 27s	remaining: 28m 2s
300:	learn: 0.6563593	test: 0.6735478	best: 0.6737635 (267)	total: 3m 35s	remaining: 26m 10s
400:	learn: 0.6610489	test: 0.6783535	best: 0.6783535 (399)	total: 4m 42s	remaining: 24m 38s
500:	learn: 0.6651746	test: 0.6790054	best: 0.6797653 (446)	total: 5m 48s	remaining: 23m 12s
600:	learn: 0.6680355	test: 0.6794071	best: 0.6801455 (548)	total: 6m 54s	remaining: 21m 50s
bestTest = 0.6801455418
bestIteration = 548
Shrink model to first 549 iterations.
CV F1 score: 0.708141

----------------- Fold 16 -----------------

Learning rate set to 0.028848
0:	learn: 0.5799234	test: 0.5798569	best: 0.5798569 (0)	total: 951ms	remaining: 39m 36s
100:	learn: 0.6407039	test: 0.6586184	best: 0.6586184 (100)	total: 1m 15s	remaining: 29m 46s
200:	learn: 0.6510267	test: 0.6675497	best: 0.6675497 (200)	total: 2m 27s	remaining: 28m 1s
300:	learn: 0.6573000	test: 0.6748675	best: 0.6749166 (299)	total: 3m 35s	remaining: 26m 11s
400:	learn: 0.6610181	test: 0.6770585	best: 0.6772317 (398)	total: 4m 42s	remaining: 24m 39s
500:	learn: 0.6647305	test: 0.6781242	best: 0.6783626 (458)	total: 5m 48s	remaining: 23m 12s
600:	learn: 0.6676640	test: 0.6803189	best: 0.6808928 (577)	total: 6m 56s	remaining: 21m 54s
700:	learn: 0.6696644	test: 0.6823360	best: 0.6824322 (695)	total: 8m 1s	remaining: 20m 35s
800:	learn: 0.6712128	test: 0.6835886	best: 0.6836925 (782)	total: 9m 8s	remaining: 19m 22s
900:	learn: 0.6728727	test: 0.6853937	best: 0.6854679 (899)	total: 10m 13s	remaining: 18m 8s
1000:	learn: 0.6740753	test: 0.6856049	best: 0.6860029 (980)	total: 11m 18s	remaining: 16m 56s
1100:	learn: 0.6753615	test: 0.6859948	best: 0.6862925 (1069)	total: 12m 23s	remaining: 15m 45s
bestTest = 0.6862925483
bestIteration = 1069
Shrink model to first 1070 iterations.
CV F1 score: 0.715237

----------------- Fold 17 -----------------

Learning rate set to 0.028848
0:	learn: 0.6220260	test: 0.6266053	best: 0.6266053 (0)	total: 859ms	remaining: 35m 47s
100:	learn: 0.6397175	test: 0.6531183	best: 0.6531183 (100)	total: 1m 15s	remaining: 29m 54s
200:	learn: 0.6502069	test: 0.6681409	best: 0.6681409 (200)	total: 2m 26s	remaining: 27m 50s
300:	learn: 0.6557266	test: 0.6744916	best: 0.6750074 (295)	total: 3m 32s	remaining: 25m 53s
400:	learn: 0.6603509	test: 0.6775177	best: 0.6775895 (398)	total: 4m 40s	remaining: 24m 25s
500:	learn: 0.6644317	test: 0.6801483	best: 0.6809972 (457)	total: 5m 46s	remaining: 23m 1s
600:	learn: 0.6674330	test: 0.6817704	best: 0.6817704 (600)	total: 6m 52s	remaining: 21m 43s
700:	learn: 0.6700047	test: 0.6823815	best: 0.6830858 (693)	total: 7m 57s	remaining: 20m 25s
800:	learn: 0.6720092	test: 0.6826762	best: 0.6832581 (741)	total: 9m 4s	remaining: 19m 14s
bestTest = 0.6832580902
bestIteration = 741
Shrink model to first 742 iterations.
CV F1 score: 0.712263

----------------- Fold 18 -----------------

Learning rate set to 0.028848
0:	learn: 0.6206366	test: 0.6208834	best: 0.6208834 (0)	total: 859ms	remaining: 35m 46s
100:	learn: 0.6399824	test: 0.6525324	best: 0.6526153 (99)	total: 1m 15s	remaining: 29m 45s
200:	learn: 0.6503631	test: 0.6657027	best: 0.6660491 (199)	total: 2m 25s	remaining: 27m 47s
300:	learn: 0.6555954	test: 0.6723759	best: 0.6731098 (285)	total: 3m 32s	remaining: 25m 53s
400:	learn: 0.6605294	test: 0.6758227	best: 0.6758227 (400)	total: 4m 39s	remaining: 24m 23s
500:	learn: 0.6641479	test: 0.6786696	best: 0.6786696 (500)	total: 5m 47s	remaining: 23m 5s
600:	learn: 0.6672262	test: 0.6800590	best: 0.6803765 (583)	total: 6m 51s	remaining: 21m 41s
700:	learn: 0.6696321	test: 0.6793247	best: 0.6805017 (625)	total: 7m 58s	remaining: 20m 27s
bestTest = 0.6805017104
bestIteration = 625
Shrink model to first 626 iterations.
CV F1 score: 0.711645

----------------- Fold 19 -----------------

Learning rate set to 0.028848
0:	learn: 0.5690103	test: 0.5718731	best: 0.5718731 (0)	total: 971ms	remaining: 40m 27s
100:	learn: 0.6389443	test: 0.6478174	best: 0.6478174 (100)	total: 1m 14s	remaining: 29m 40s
200:	learn: 0.6499817	test: 0.6630749	best: 0.6631008 (194)	total: 2m 25s	remaining: 27m 47s
300:	learn: 0.6563406	test: 0.6716956	best: 0.6717956 (296)	total: 3m 33s	remaining: 25m 58s
400:	learn: 0.6602187	test: 0.6755996	best: 0.6756276 (398)	total: 4m 40s	remaining: 24m 27s
500:	learn: 0.6642199	test: 0.6780608	best: 0.6785488 (495)	total: 5m 47s	remaining: 23m 7s
600:	learn: 0.6670488	test: 0.6790586	best: 0.6792059 (555)	total: 6m 53s	remaining: 21m 47s
700:	learn: 0.6696388	test: 0.6797475	best: 0.6798443 (698)	total: 7m 59s	remaining: 20m 30s
800:	learn: 0.6712782	test: 0.6821318	best: 0.6824627 (789)	total: 9m 4s	remaining: 19m 15s
900:	learn: 0.6726262	test: 0.6819789	best: 0.6829620 (837)	total: 10m 9s	remaining: 18m 1s
bestTest = 0.6829620071
bestIteration = 837
Shrink model to first 838 iterations.
CV F1 score: 0.710164
	F1 score: 0.710289
```

